In [1]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Demand Forecasting - Exploratory Analysis\n",
    "\n",
    "Este notebook realiza un análisis exploratorio del sistema de pronóstico de demanda."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "import sys\n",
    "sys.path.append('..')\n",
    "\n",
    "import pandas as pd\n",
    "import numpy as np\n",
    "import matplotlib.pyplot as plt\n",
    "import seaborn as sns\n",
    "\n",
    "from src.forecasting.data import generate_synthetic_demand, load_data\n",
    "from src.forecasting.features import create_all_features, get_feature_columns\n",
    "from src.forecasting.models import DemandForecaster\n",
    "from src.forecasting.evaluate import calculate_metrics\n",
    "\n",
    "plt.style.use('seaborn-v0_8-darkgrid')\n",
    "%matplotlib inline"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 1. Generación y Visualización de Datos"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "df = generate_synthetic_demand(periods=730)\n",
    "print(f\"Shape: {df.shape}\")\n",
    "df.head()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "fig, axes = plt.subplots(2, 2, figsize=(15, 10))\n",
    "\n",
    "axes[0, 0].plot(df['date'], df['demand'])\n",
    "axes[0, 0].set_title('Serie Temporal Completa')\n",
    "axes[0, 0].set_xlabel('Fecha')\n",
    "axes[0, 0].set_ylabel('Demanda')\n",
    "\n",
    "df['demand'].hist(bins=50, ax=axes[0, 1])\n",

SyntaxError: incomplete input (3759801978.py, line 66)